<a href="https://colab.research.google.com/github/Surya-Uday-Singh/CNN_From_Scratch-MNIST-/blob/main/MnistCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class Conv:
  #This defines a convolutional layer with 3x3 filters

  def __init__(self, num_of_filters):
    self.num_of_filters = num_of_filters

    #This creates a defined 3x3 filter matrice with random numbers
    self.filters = np.random.randn(num_of_filters, 3, 3)/9

  def filter_regions(self, img):

    h, w = img.shape

    for i in range(h - 2):
      for j in range(w - 2):
        img_region = img[i:(i + 3), j:(j + 3)]
        yield img_region, i, j

  def forward_propagate(self, input):

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_of_filters))

    for img_region, i, j in self.filter_regions(input):
      output[i, j] = np.sum(img_region * self.filters, axis=(1,2))

    return output


